# CIS + 2560 量能確認選股

基於 CIS 選股策略，加入 2560 戰法的量能確認濾網，過濾盤整期間的假買點。

## 選股條件

### CIS逆買（領先/逆勢買點）
1. 股價在 25MA 之下
2. 25MA「走平」或「向上」
3. 5MA 向上
4. 連續出現陽線
5. **[2560] 5日均量 > 60日均量（量能確認）**

### CIS買（順勢買點）
1. 25MA「走平」或「向上」
2. 5MA 位於 25MA 之上（多頭排列）
3. 股價位於 5MA 之上
4. **[2560] 5日均量 > 60日均量（量能確認）**

### CIS賣（賣出信號）
1. 5MA 向下
2. 股價跌破 5MA
- ⚠️ 賣出信號不受量能過濾影響，避免錯過停損時機

### 篩選條件
- 5日平均成交量 > 1000張
- 篩出結果以「相對5日平均成交量倍數」由大到小排列

### 2560 量能過濾說明
- 核心概念：「量先價行」— 5日均量 > 60日均量代表真實買盤介入，反之為「誘多」陷阱
- 可透過 `ENABLE_VOLUME_CONFIRMATION` 開關切換（`True` 啟用 / `False` 停用）

In [ ]:
import os
import sys
from datetime import date
from datetime import datetime
from pathlib import Path

import pandas as pd
from lightweight_charts import JupyterChart

In [ ]:
from finlab import data
import finlab

In [ ]:
# 引用自建公用模組
sys.path.insert(0, str(Path.cwd().parent))
from proj_util_pkg.settings import ProjEnvSettings

from proj_util_pkg.finlab_api import finlab_manager as flm
from proj_util_pkg.google_api import gspread_manager as gsm
from proj_util_pkg.common import tw_stock_topic as tst

## 公用參數設定

In [ ]:
# finlab api 服務初始化
finlab = flm.FinlabManager()
data.force_cloud_download = False

In [ ]:
# 資訊輸出Google SpreadSheet 表單參數設定
GSPERAD_SHEET_KEY = os.environ.get('gspread_wb_key')
OUTPUT_GSHEET_NAME = '選股清單06'

In [ ]:
# 本地報表輸出路徑
REPORT_PATH = os.environ.get('report_path')

## 外部資料讀取

In [ ]:
# 讀取台股資訊
close = data.get("price:收盤價", save_to_storage=True)
open_price = data.get("price:開盤價", save_to_storage=True)
high = data.get("price:最高價", save_to_storage=True)
low = data.get("price:最低價", save_to_storage=True)
vol = data.get("price:成交股數", save_to_storage=True)
stock_info = data.get('company_basic_info', save_to_storage=True)
pe_ratio = data.get('price_earning_ratio:本益比', save_to_storage=True)
pb_ratio = data.get('price_earning_ratio:股價淨值比', save_to_storage=True)

## CIS 輔助函數定義

In [ ]:
# ===== CIS 交易策略指標 - 輔助函數定義 =====

def is_ma_flat(ma_series, threshold=0.003, periods=3):
    """
    判斷均線是否走平
    條件：連續 periods 日，均線日變動率的絕對值都小於 threshold (0.3%)
    """
    ma_change_rate = ma_series.pct_change(fill_method=None).abs()
    is_flat = (ma_change_rate < threshold).rolling(window=periods).min() == 1
    return is_flat


def is_ma_up(ma_series):
    """
    判斷均線是否向上
    條件：今日均線 > 昨日均線
    """
    return ma_series > ma_series.shift(1)


def is_ma_down(ma_series):
    """
    判斷均線是否向下
    條件：今日均線 < 昨日均線
    """
    return ma_series < ma_series.shift(1)


def is_consecutive_bullish(close, open_price, periods=2):
    """
    判斷是否連續出現陽線
    條件：連續 periods 日，收盤價 > 開盤價
    """
    is_bullish = close > open_price
    return is_bullish.rolling(window=periods).min() == 1


print("CIS 輔助函數定義完成")

## CIS 信號計算

In [ ]:
# ===== 計算移動平均線 =====
sma5 = close.average(5)
sma25 = close.average(25)
vol_sma5 = vol.average(5)
vol_sma60 = vol.average(60)  # 60日成交量均線（2560策略基準量）

print("移動平均線計算完成")
print(f"SMA5 形狀: {sma5.shape}")
print(f"SMA25 形狀: {sma25.shape}")
print(f"Vol_SMA5 形狀: {vol_sma5.shape}")
print(f"Vol_SMA60 形狀: {vol_sma60.shape}")

In [ ]:
# ===== 2560 量能確認條件 =====
# 核心概念：5日均量 > 60日均量 = 真實買盤，反之為誘多陷阱
# 設定開關，方便對比有無量能過濾的差異
ENABLE_VOLUME_CONFIRMATION = True

cond_vol_confirm = vol_sma5 > vol_sma60  # 5日均量 > 60日均量

print(f"量能確認過濾: {'啟用' if ENABLE_VOLUME_CONFIRMATION else '停用'}")


# ===== CIS逆買（領先/逆勢買點）=====
# 條件：
# 1. 股價在 25MA 之下
# 2. 25MA「走平」或「向上」
# 3. 5MA 向上
# 4. 連續出現陽線
# 5. [2560] 5日均量 > 60日均量（量能確認）

cond_reverse_buy_1 = close < sma25  # 股價在25MA之下
cond_reverse_buy_2 = is_ma_flat(sma25) | is_ma_up(sma25)  # 25MA走平或向上
cond_reverse_buy_3 = is_ma_up(sma5)  # 5MA向上
cond_reverse_buy_4 = is_consecutive_bullish(close, open_price)  # 連續陽線

signal_cis_reverse_buy = (
    cond_reverse_buy_1 &
    cond_reverse_buy_2 &
    cond_reverse_buy_3 &
    cond_reverse_buy_4
)

# 套用量能過濾
if ENABLE_VOLUME_CONFIRMATION:
    signal_cis_reverse_buy = signal_cis_reverse_buy & cond_vol_confirm

print("CIS逆買信號計算完成")


# ===== CIS買（順勢買點）=====
# 條件：
# 1. 25MA「走平」或「向上」
# 2. 5MA 位於 25MA 之上（多頭排列）
# 3. 股價位於 5MA 之上
# 4. [2560] 5日均量 > 60日均量（量能確認）

cond_buy_1 = is_ma_flat(sma25) | is_ma_up(sma25)  # 25MA走平或向上
cond_buy_2 = sma5 > sma25  # 5MA在25MA之上（多頭排列）
cond_buy_3 = close > sma5  # 股價在5MA之上

signal_cis_buy = cond_buy_1 & cond_buy_2 & cond_buy_3

# 套用量能過濾
if ENABLE_VOLUME_CONFIRMATION:
    signal_cis_buy = signal_cis_buy & cond_vol_confirm

print("CIS買信號計算完成")


# ===== CIS賣（賣出信號）=====
# 條件（兩者皆要）：
# 1. 5MA 向下
# 2. 股價跌破 5MA
# 注意：賣出信號不套用量能過濾，避免錯過停損時機

cond_sell_1 = is_ma_down(sma5)  # 5MA向下
cond_sell_2 = close < sma5  # 股價跌破5MA

signal_cis_sell = cond_sell_1 & cond_sell_2

print("CIS賣信號計算完成")

## CIS 逆買 & CIS 買 選股篩選

In [ ]:
# ===== 分析開始時間紀錄 =====
start_time = datetime.now()
print(f"ANA001_台股選股 「CIS選股」 分析開始時間: {start_time}")

# ===== 篩選最新交易日出現 CIS逆買 或 CIS買 的股票 =====
# 合併兩個買入信號
signal_cis_any_buy = signal_cis_reverse_buy | signal_cis_buy

# 取最新一個交易日
latest_signal = signal_cis_any_buy.tail(1)
latest_date = latest_signal.index[0]

# 篩選出有信號的股票代號
filtered_symbols = latest_signal.columns[latest_signal.iloc[0]].tolist()

print(f"最新交易日: {latest_date.strftime('%Y-%m-%d')}")
print(f"CIS買入信號股票數（篩選前）: {len(filtered_symbols)}")

# ===== 5日平均成交量篩選 > 1000張 =====
vol_filter = (vol_sma5 / 1000) > 1000  # 成交股數轉張數，篩選5日均量 > 1000張
latest_vol_filter = vol_filter.tail(1)
vol_qualified_symbols = latest_vol_filter.columns[latest_vol_filter.iloc[0]].tolist()

# 取交集：同時符合信號 AND 成交量條件的股票
qualified_symbols = list(set(filtered_symbols) & set(vol_qualified_symbols))

print(f"成交量篩選後股票數: {len(qualified_symbols)}")

In [ ]:
# ===== 建立篩選結果 DataFrame =====
df_filtered = pd.DataFrame({'symbol': qualified_symbols})

# --- 股票名稱 ---
stock_name = stock_info[['stock_id', '公司簡稱']].copy()
stock_name = stock_name.rename(columns={'stock_id': 'symbol'})

# --- 最新收盤價 ---
last_close = close.tail(1).T.reset_index()
last_close.columns = ['symbol', '收盤價']

# --- 當日成交量（張）---
last_vol = vol.tail(1).T.reset_index()
last_vol.columns = ['symbol', 'vol_raw']
last_vol['vol_raw'] = last_vol['vol_raw'].fillna(0)
last_vol['成交量(張)'] = (last_vol['vol_raw'] / 1000).round().astype(int)

# --- 5日平均成交量（張）---
last_vol_sma5 = vol_sma5.tail(1).T.reset_index()
last_vol_sma5.columns = ['symbol', 'vol_sma5_raw']
last_vol_sma5['vol_sma5_raw'] = last_vol_sma5['vol_sma5_raw'].fillna(0)
last_vol_sma5['五日均量(張)'] = (last_vol_sma5['vol_sma5_raw'] / 1000).round().astype(int)

# --- 60日平均成交量（張）---
last_vol_sma60 = vol_sma60.tail(1).T.reset_index()
last_vol_sma60.columns = ['symbol', 'vol_sma60_raw']
last_vol_sma60['vol_sma60_raw'] = last_vol_sma60['vol_sma60_raw'].fillna(0)
last_vol_sma60['六十日均量(張)'] = (last_vol_sma60['vol_sma60_raw'] / 1000).round().astype(int)

# --- 本益比 ---
last_pe = pe_ratio.tail(1).T.reset_index()
last_pe.columns = ['symbol', '本益比']

# --- 股價淨值比 ---
last_pb = pb_ratio.tail(1).T.reset_index()
last_pb.columns = ['symbol', '股價淨值比']

# --- 信號類型判定 ---
latest_reverse_buy = signal_cis_reverse_buy.tail(1)
latest_buy = signal_cis_buy.tail(1)

def get_signal_type(symbol):
    """判斷股票的CIS信號類型"""
    signals = []
    if symbol in latest_reverse_buy.columns and latest_reverse_buy[symbol].iloc[0]:
        signals.append('CIS逆買')
    if symbol in latest_buy.columns and latest_buy[symbol].iloc[0]:
        signals.append('CIS買')
    return ', '.join(signals)

df_filtered['信號類型'] = df_filtered['symbol'].apply(get_signal_type)

# ===== 合併所有欄位 =====
merged_df = df_filtered.merge(stock_name, on='symbol', how='left')
merged_df = merged_df.merge(last_close, on='symbol', how='left')
merged_df = merged_df.merge(last_vol, on='symbol', how='left')
merged_df = merged_df.merge(last_vol_sma5, on='symbol', how='left')
merged_df = merged_df.merge(last_vol_sma60, on='symbol', how='left')
merged_df = merged_df.merge(last_pe, on='symbol', how='left')
merged_df = merged_df.merge(last_pb, on='symbol', how='left')

# ===== 計算 量能倍數（5日均量 / 60日均量）=====
merged_df['量能倍數'] = (
    merged_df['五日均量(張)'] / merged_df['六十日均量(張)'].replace(0, float('nan'))
).round(2)

# ===== 計算 相對5日平均成交量倍數 =====
merged_df['相對5日平均成交量倍數'] = (
    merged_df['成交量(張)'] / merged_df['五日均量(張)']
).round(2)

# ===== 排序：相對5日平均成交量倍數 由大到小 =====
merged_df = merged_df.sort_values('相對5日平均成交量倍數', ascending=False).reset_index(drop=True)

# ===== 加入看盤連結和題材概念股 =====
merged_df['web_link'] = merged_df['symbol'].apply(
    lambda x: f"https://www.wantgoo.com/stock/{x}/technical-chart"
)
merged_df['題材概念股'] = merged_df['symbol'].apply(lambda x: tst.read_topic_stocks(x))

# ===== 清理臨時欄位，重新排列輸出欄位 =====
output_columns = [
    'symbol', '公司簡稱', '收盤價', '成交量(張)', '五日均量(張)',
    '六十日均量(張)', '量能倍數',
    '相對5日平均成交量倍數', '信號類型', '本益比', '股價淨值比',
    'web_link', '題材概念股'
]
merged_df = merged_df[[col for col in output_columns if col in merged_df.columns]]

# 重命名欄位為最終輸出格式
merged_df = merged_df.rename(columns={'symbol': '股票代號', '公司簡稱': '股票名稱'})

print(f"最終篩選結果: {len(merged_df)} 檔股票")
merged_df

In [ ]:
# 輸出報表留存
today = datetime.now().strftime("%Y%m%d")
merged_df.to_excel(f'{REPORT_PATH}/選股06_{today}.xlsx', index=False)
print(f"報表已儲存至: {REPORT_PATH}/選股06_{today}.xlsx")

## 輸出結果至Google Sheet

In [ ]:
# Google SpreadSheet 公用程式初始化
gspread_mgr = gsm.GspreadManager()
gspread_wb = gspread_mgr.get_spreadsheet(GSPERAD_SHEET_KEY)

print(f"更新Google 表單：{gspread_wb.title}，工作表：{OUTPUT_GSHEET_NAME}")

In [ ]:
# 刪除再重建工作表
gspread_mgr.recreate_worksheet(GSPERAD_SHEET_KEY, OUTPUT_GSHEET_NAME)

In [ ]:
# 更新工作表資料
# 將NaN值轉換為空字串，避免上傳時出錯
output_df = merged_df.fillna('')

gspread_mgr.update_worksheet_values(
    GSPERAD_SHEET_KEY,
    OUTPUT_GSHEET_NAME,
    [output_df.columns.values.tolist()] + output_df.values.tolist()
)

print(f"已成功更新 {len(output_df)} 筆資料至 Google SpreadSheet")

In [ ]:
# 計算總執行時間
end_time = datetime.now()
total_time = end_time - start_time

print(f"分析結束時間: {end_time}")
print(f"總執行時間: {total_time}")

---
## 信號驗證區塊 - K線圖與CIS信號標示

以下區塊用於驗證 CIS 信號的正確性，透過 K 線圖視覺化呈現買賣信號。

In [ ]:
# ===== 驗證用股票代碼設定 =====
VERIFY_SYMBOL = '00635U'

print(f"驗證股票代碼：{VERIFY_SYMBOL}")

In [ ]:
# ===== 取指定股票代碼 - 近2年信號輸出 =====

# 計算日期範圍（近2年）
end_date = close.index[-1]
start_date = end_date - pd.DateOffset(years=2)

# 篩選日期範圍
mask = (close.index >= start_date) & (close.index <= end_date)

# 建立驗證表格
result_df = pd.DataFrame({
    '日期': close.index[mask],
    '收盤價': close[VERIFY_SYMBOL][mask].values,
    '開盤價': open_price[VERIFY_SYMBOL][mask].values,
    '5MA': sma5[VERIFY_SYMBOL][mask].round(2).values,
    '25MA': sma25[VERIFY_SYMBOL][mask].round(2).values,
    '5日均量': (vol_sma5[VERIFY_SYMBOL][mask] / 1000).round().values,
    '60日均量': (vol_sma60[VERIFY_SYMBOL][mask] / 1000).round().values,
    '量能確認': (vol_sma5[VERIFY_SYMBOL][mask] > vol_sma60[VERIFY_SYMBOL][mask]).values,
    'CIS買': signal_cis_buy[VERIFY_SYMBOL][mask].values,
    'CIS逆買': signal_cis_reverse_buy[VERIFY_SYMBOL][mask].values,
    'CIS賣': signal_cis_sell[VERIFY_SYMBOL][mask].values
})

# 篩選有信號的日期
signal_df = result_df[
    result_df['CIS買'] |
    result_df['CIS逆買'] |
    result_df['CIS賣']
].copy()

# 新增信號類型欄位
def get_signal_type_for_verify(row):
    signals = []
    if row['CIS買']:
        signals.append('CIS買')
    if row['CIS逆買']:
        signals.append('CIS逆買')
    if row['CIS賣']:
        signals.append('CIS賣')
    return ', '.join(signals)

signal_df['信號類型'] = signal_df.apply(get_signal_type_for_verify, axis=1)

# ===== 連續相同信號只取第一個 =====
signal_df = signal_df.reset_index()
signal_df.rename(columns={'index': '原始索引'}, inplace=True)

signal_df['前一索引'] = signal_df['原始索引'].shift(1)
signal_df['前一信號'] = signal_df['信號類型'].shift(1)
signal_df['是連續交易日'] = (signal_df['原始索引'] - signal_df['前一索引']) == 1
signal_df['信號類型相同'] = signal_df['信號類型'] == signal_df['前一信號']
signal_df['是首次信號'] = ~(signal_df['是連續交易日'] & signal_df['信號類型相同'])

first_signal_df = signal_df[signal_df['是首次信號']].copy()
first_signal_df = first_signal_df.drop(columns=['原始索引', '前一索引', '前一信號', '是連續交易日', '信號類型相同', '是首次信號'])

# 輸出統計資訊
print(f"===== {VERIFY_SYMBOL} CIS + 2560 交易信號統計（{start_date.strftime('%Y-%m-%d')} ~ {end_date.strftime('%Y-%m-%d')}）=====")
print(f"量能確認過濾: {'啟用' if ENABLE_VOLUME_CONFIRMATION else '停用'}")
print(f"總交易日數: {len(result_df)}")
print(f"原始信號總數: {len(signal_df)}")
print(f"去除連續重複後信號數: {len(first_signal_df)}")
print()
print("各類信號次數（去除連續重複後）:")
print(f"  CIS買: {(first_signal_df['信號類型'] == 'CIS買').sum()}")
print(f"  CIS逆買: {(first_signal_df['信號類型'] == 'CIS逆買').sum()}")
print(f"  CIS賣: {(first_signal_df['信號類型'] == 'CIS賣').sum()}")
print()

# 輸出信號表格
verify_output_columns = ['日期', '收盤價', '開盤價', '5MA', '25MA', '5日均量', '60日均量', '量能確認', '信號類型']
first_signal_df[verify_output_columns]

In [ ]:
# ===== 量能過濾效果對比 =====
# 計算原始信號（不含量能過濾）
original_reverse_buy = (
    cond_reverse_buy_1 &
    cond_reverse_buy_2 &
    cond_reverse_buy_3 &
    cond_reverse_buy_4
)
original_buy = cond_buy_1 & cond_buy_2 & cond_buy_3
original_any_buy = original_reverse_buy | original_buy

# 對比指定驗證股票
mask_verify = (close.index >= start_date) & (close.index <= end_date)

orig_buy_count = original_buy[VERIFY_SYMBOL][mask_verify].sum()
orig_reverse_buy_count = original_reverse_buy[VERIFY_SYMBOL][mask_verify].sum()
orig_total = original_any_buy[VERIFY_SYMBOL][mask_verify].sum()

filtered_buy_count = signal_cis_buy[VERIFY_SYMBOL][mask_verify].sum()
filtered_reverse_buy_count = signal_cis_reverse_buy[VERIFY_SYMBOL][mask_verify].sum()
filtered_total = (signal_cis_reverse_buy | signal_cis_buy)[VERIFY_SYMBOL][mask_verify].sum()

print(f"===== {VERIFY_SYMBOL} 量能過濾效果對比（{start_date.strftime('%Y-%m-%d')} ~ {end_date.strftime('%Y-%m-%d')}）=====")
print(f"量能確認過濾: {'啟用' if ENABLE_VOLUME_CONFIRMATION else '停用'}")
print()
print(f"{'信號類型':<12} {'原始信號日數':>10} {'過濾後信號日數':>12} {'過濾日數':>8} {'過濾比例':>8}")
print(f"{'-'*54}")
print(f"{'CIS買':<12} {orig_buy_count:>10} {filtered_buy_count:>12} {orig_buy_count - filtered_buy_count:>8} {((orig_buy_count - filtered_buy_count) / max(orig_buy_count, 1) * 100):>7.1f}%")
print(f"{'CIS逆買':<12} {orig_reverse_buy_count:>10} {filtered_reverse_buy_count:>12} {orig_reverse_buy_count - filtered_reverse_buy_count:>8} {((orig_reverse_buy_count - filtered_reverse_buy_count) / max(orig_reverse_buy_count, 1) * 100):>7.1f}%")
print(f"{'-'*54}")
print(f"{'合計':<12} {orig_total:>10} {filtered_total:>12} {orig_total - filtered_total:>8} {((orig_total - filtered_total) / max(orig_total, 1) * 100):>7.1f}%")

In [ ]:
# ===== 建立 K 線圖資料 =====

# 建立 OHLC DataFrame（包含成交量）
ohlc_df = pd.DataFrame({
    'time': close.index[mask],
    'open': open_price[VERIFY_SYMBOL][mask].values,
    'high': high[VERIFY_SYMBOL][mask].values,
    'low': low[VERIFY_SYMBOL][mask].values,
    'close': close[VERIFY_SYMBOL][mask].values,
    'volume': vol[VERIFY_SYMBOL][mask].values
})

# 計算均線
ohlc_df['MA5'] = ohlc_df['close'].rolling(window=5).mean()
ohlc_df['MA25'] = ohlc_df['close'].rolling(window=25).mean()

print(f"===== {VERIFY_SYMBOL} OHLC 資料（近2年）=====")
print(f"資料期間: {ohlc_df['time'].min().strftime('%Y-%m-%d')} ~ {ohlc_df['time'].max().strftime('%Y-%m-%d')}")
print(f"資料筆數: {len(ohlc_df)}")

In [ ]:
# ===== 繪製 K 線圖並標示 CIS 信號 =====

# 建立圖表
chart = JupyterChart(height=600, width=1200)

# 設定紅漲綠跌
chart.candle_style(
    up_color='#982e2e',      # 漲 - 紅色
    down_color='#205f0b',    # 跌 - 綠色
)

# 設定成交量柱狀圖顏色（紅漲綠跌）
chart.volume_config(
    up_color='#982e2e',      # 漲 - 紅色
    down_color='#205f0b',    # 跌 - 綠色
)

# 設定 OHLC 資料（包含成交量）
chart.set(ohlc_df)

# 新增 5MA 均線 (藍色)
ma5_line = chart.create_line(name='MA5', color='#2196F3', width=2, price_label=False)
ma5_data = ohlc_df[['time', 'MA5']].dropna()
ma5_line.set(ma5_data)

# 新增 25MA 均線 (橘色)
ma25_line = chart.create_line(name='MA25', color='#FF9800', width=2, price_label=False)
ma25_data = ohlc_df[['time', 'MA25']].dropna()
ma25_line.set(ma25_data)

# 自訂時間格式為 YYYY/MM/DD
chart.run_script(f'''
    {chart.id}.chart.applyOptions({{
        localization: {{
            timeFormatter: (time) => {{
                const date = new Date(time * 1000);
                const year = date.getFullYear();
                const month = String(date.getMonth() + 1).padStart(2, '0');
                const day = String(date.getDate()).padStart(2, '0');
                return year + '/' + month + '/' + day;
            }}
        }},
        timeScale: {{
            tickMarkFormatter: (time) => {{
                const date = new Date(time * 1000);
                const year = date.getFullYear();
                const month = String(date.getMonth() + 1).padStart(2, '0');
                const day = String(date.getDate()).padStart(2, '0');
                return year + '/' + month + '/' + day;
            }}
        }}
    }});
''')

# 啟用 legend
chart.legend(
    visible=True,
    ohlc=True,
    percent=True,
    color='rgb(191, 195, 203)',
    font_size=12,
    color_based_on_candle=True
)

# ===== 根據 first_signal_df 標示 CIS 信號 =====
# 信號顏色與樣式設定
signal_config = {
    'CIS買': {'position': 'below', 'shape': 'arrow_up', 'color': '#2196F3', 'text': 'CIS買'},      # 藍色向上箭頭
    'CIS逆買': {'position': 'below', 'shape': 'arrow_up', 'color': '#9C27B0', 'text': 'CIS逆買'},  # 紫色向上箭頭
    'CIS賣': {'position': 'above', 'shape': 'arrow_down', 'color': '#FF5722', 'text': 'CIS賣'},    # 橘紅色向下箭頭
}

# 遍歷信號並標示
for _, row in first_signal_df.iterrows():
    signal_type = row['信號類型']
    signal_date = row['日期']
    
    # 處理複合信號（如 "CIS買, CIS逆買"）
    for sig_type, config in signal_config.items():
        if sig_type in signal_type:
            chart.marker(
                time=pd.Timestamp(signal_date),
                position=config['position'],
                shape=config['shape'],
                color=config['color'],
                text=config['text']
            )

print(f"===== {VERIFY_SYMBOL} K線圖 CIS 信號標示 =====")
print(f"已標示 {len(first_signal_df)} 個 CIS 信號")
print(f"   - CIS買 (藍色↑): {(first_signal_df['信號類型'] == 'CIS買').sum()} 個")
print(f"   - CIS逆買 (紫色↑): {(first_signal_df['信號類型'] == 'CIS逆買').sum()} 個")
print(f"   - CIS賣 (橘紅色↓): {(first_signal_df['信號類型'] == 'CIS賣').sum()} 個")

chart.load()